## Interview Prompt

### Objective
Develop a personalized ranking algorithm for users on thredUP, focusing on maximizing purchases by considering user-item interaction history and item attributes.  Output should be a re-ranking of search_results.csv that is personalized for each user.

### Dataset Overview
<b>user_item_interact.csv</b>: Data for 500 users, covering the last 6 months of interactions with items.<br>
<b>item_information.csv</b>: Data with information about items.<br>
<b>search_results.csv</b>: Unranked results from a search.

Here are descriptions of the data provided in the files:
<b>user_id</b> - unique identifier for the user<br>
<b>item_number</b> - unique identifier for the item<br>
<b>num_views</b> - number of times the user viewed the item<br>
<b>num_atc</b> - number of times the user added the item to cart (etc)<br>
<b>favorited</b> - whether or not the user added the item to their favorites (a way to bookmark the item for finding later)<br>
<b>purchased</b> - (boolean) means that the user purchased the item<br>
<b>returned</b> - (boolean) means that the user purchased and returned the item<br>

<b>items_listed_date</b> - date the item was listed on thredup.com<br>
<b>item_category_name</b> - item category<br>
<b>brand_id</b> - unique identifier for the item's brand<br>

<b>brand_tier_score</b> - item brand tier (higher number means higher-end brand)<br>
<b>item_prices_msrp</b> - manufacturer suggested retail for the price when new<br>
<b>item_listing_price</b> - price the item was first listed at<br>
<b>item_current_price</b>	- price the item is currently listed at (usually older items are discounted more)<br>
<b>items_purchased_date</b> - date the item was purchased (if it was purchased)<br>
<b>item_purchase_price</b> - price the item was purchased at (if it was purchased)<br>
<b>item_primary_color</b> - the primary color of the item<br>
<b>item_pattern</b> - the primary pattern of the item<br>
<b>seasonality_winter</b> - (boolean) indicator if the item is an item worn in winter<br>
<b>seasonality_summer</b>- (boolean) indicator if the item is an item worn in summer<br>
<b>seasonality_spring</b> - (boolean) indicator if the item is an item worn in spring<br>
<b>seasonality_fall</b> - (boolean) indicator if the item is an item worn in fall<br>
<b>item_score</b> - a score given to the item (higher score means more likely to sell)<br>
<b>favorite_count</b> - the total number of users who have favorited the item<br>

### Evaluation Metric
Conversion rate (i.e., the rate of purchases).

### Technical Requirements:
Language: Python<br>
Libraries: Open choice

In [ ]:
import pandas as pd
import numpy as np

# Load the provided datasets
user_item_interactions_path = './user_item_interact.csv'
search_data_path = './search_results.csv'
item_information_path = './item_information.csv'

# Reading the datasets
user_item_df = pd.read_csv(user_item_interactions_path)
search_df = pd.read_csv(search_data_path)
item_df = pd.read_csv(item_information_path)

In [20]:
user_item_df

,user_id,item_number,num_views,num_atc,favorited,purchased,returned
0,1,AZ47II7IL,5,1,1,1,0
1,1,A1Z7I944L,4,1,1,1,0
2,1,A1EA1A9AI,6,1,1,1,1
3,1,AFIA9L7A7,4,1,1,1,0
4,1,A14A1LZ9I,2,0,1,0,0
...,...,...,...,...,...,...,...
128768,500,AFI9ZEIE7,2,0,0,0,0
128769,500,A17ALFEIF,1,0,0,0,0
128770,500,A119ELEIE,1,0,0,0,0
128771,500,AFI97A77Z,1,0,0,0,0


In [23]:
item_df

,item_number,items_listed_date,item_category_name,brand_id,brand_tier_score,item_prices_msrp,item_listing_price,item_current_price,items_purchased_date,item_purchase_price,item_primary_color,item_pattern,seasonality_winter,seasonality_summer,seasonality_spring,seasonality_fall,item_score,favorite_count
0,A1FZ117EI,5/31/23,'3/4 Sleeve Jersey',2832,3.0,57.0,23.99,19.99,9/7/23,21.99,Purple,Solid,1,1,1,1,NaN,38
1,A1I7FZFLI,3/7/23,'3/4 Sleeve Turtleneck',380,4.0,21.0,7.99,6.99,8/19/23,7.20,Blue,Solid,1,0,1,1,NaN,8
2,AFA1AA4LI,9/10/23,'Active Skirt',479,4.0,41.0,16.99,14.99,10/6/23,11.87,Black,Solid,0,1,1,1,NaN,5
3,AZ44FFI97,12/24/22,'Active Skirt',955,2.0,14.0,8.99,6.99,10/12/23,6.45,Black,NaN,0,1,1,1,NaN,5
4,A14LFI4F9,6/9/23,'Belt Bag',4,4.0,76.0,26.99,26.99,7/12/23,28.99,Black,Solid,1,1,1,1,NaN,46
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
252139,AF9FFI11F,12/28/23,'Short Sleeve T-Shirt',13544,1.5,19.0,8.74,8.74,12/29/23,10.99,Red,NaN,0,1,1,0,NaN,1
252140,AF9FF4Z7L,12/28/23,'Cocktail Dress',13545,2.0,64.0,19.74,19.74,NaN,NaN,Blue,Solid,0,0,1,1,0.322811,3
252141,AF9F71EA4,12/22/23,'Casual Skirt',13546,5.0,95.0,19.74,19.74,NaN,NaN,Tan,Tweed,0,1,1,0,0.561710,7
252142,AF9F9F99A,12/29/23,'Cocktail Dress',13548,1.0,32.0,10.74,10.74,12/29/23,12.99,Black,Solid,0,0,1,1,NaN,6


In [24]:
search_df

,item_number
0,AF4AEA74F
1,AFFZLL9L7
2,AF44LEE4F
3,AF1FA9I17
4,AF71EEEZF
...,...
9995,AFFLZFL11
9996,AFE9E7A44
9997,AF7IA9941
9998,AF91A1ZF1
